# Pronóstico adaptativo

## Descripción del problema real

Los pronósticos de los precios de la electricidad en mercados liberalizados son un insumo fundamental para la toma de decisiones dentro de las organizaciones. Fundamentalmente, los pronosticos de corto plazo son utilizados en decisiones de carácter operativo. En el caso abordado, es necesario contar con los pronósticos para definir la politica de operación de una empresa del sector eléctrico.

## Descripción del problema en términos de los datos

La carpeta `datos/precios/` contiene los precios historicos horarios de la electricidad en la Bolsa de Energía del mercado eléctrico colombiano, publicados por el operador del mercado. Se desean obtener pronósticos para el precio promedio diario para los próximos siete (7) días a la fecha actual.

## Aproximaciones posibles

En este caso, se desea evaluar la capacidad de pronóstico de un ADALINE con aprendizaje en línea.

## Requerimientos

Usted debe:

* Procesar los datos históricos para conformar el conjunto de datos. Los archivos de Excel no pueden ser modificados y actualizados directamente por el operador del sistema. Su código debe leer los archivos y crear la serie de precios promedio diarios de la electricidad.


* Determinar si el modelo debe pronosticar los precios promedios sin ninguna transformación, o si la inclusión de una transformación (logaritmo natural, raíz cúbica, raíz cuadrada, etc) resulta en un incremento de la precisión del pronóstico.


* Generar el pronóstico para los precios de los próximos siete días.


* Preparar el código para que el modelo sea entrenado usando el gradiente y el momentum.


* Determinar cuál es el número óptimo de retardos (observaciones) que el modelo debe considerar hacia atrás para producir el pronóstico.


* Determinar los valores óptimos de la tasa de aprendizaje y el momentum.


# Se importan las librerias a utilizar a lo largo de todo el código

In [9]:
import pandas as pd
import numpy as np
import glob

# Lectura de datos de los precios desde 1995 hasta 2018

In [10]:
!ls ./datos/precios/

'Precio_Bolsa_Nacional_($kwh)_1995.xlsx'
'Precio_Bolsa_Nacional_($kwh)_1996.xlsx'
'Precio_Bolsa_Nacional_($kwh)_1997.xlsx'
'Precio_Bolsa_Nacional_($kwh)_1998.xlsx'
'Precio_Bolsa_Nacional_($kwh)_1999.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2000.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2001.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2002.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2003.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2004.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2005.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2006.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2007.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2008.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2009.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2010.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2011.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2012.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2013.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2014.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2015.xlsx'
'Precio_Bolsa_Nacional_($kwh)_2016.xls'
'Precio_Bolsa_Nacional_($kwh)_2017.xls'
'Precio_Bolsa_Nacional_($kwh)_2018.xls'


In [16]:
datos = pd.DataFrame(list(df.columns("./datos/precios/'Precio_Bolsa_Nacional_($kwh)_1995.xlsx")))
for f in sorted(glob.glob("./datos/precios/*")):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)

NameError: name 'df' is not defined